In [1]:
%pip install --quiet pydantic chromadb langchain langchain_experimental langchain_openai langchain_community, langchain_chroma, langchainhub

DEPRECATION: Loading egg at /home/solomon/.local/lib/python3.11/site-packages/openapi_client-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Invalid requirement: 'langchain_community,'
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from pathlib import Path
import json

product_path = Path("../initial-training-sets/datasets/dataset_products.json")
product_images_path = Path("../initial-training-sets/datasets/images/")
product_data = json.loads(product_path.read_text())

from datetime import date
from uuid import UUID
from pydantic import BaseModel
from typing import List

class RawReview(BaseModel):
    id: UUID
    review_ref: str
    product_id: UUID
    review_content: str
    review_title: str
    date_written: date
    product_asin: str
    helpful_count: int
    rating_given: int
    review_page_url: str

class RAWProduct(BaseModel):
    id: UUID
    name: str
    description: str
    product_asin: str
    overall_ratings: float
    total_customers_that_rated: int
    price: float
    currency: str
    category: str
    sub_category: str
    product_page_url: str
    image_url: str
    reviews: List[RawReview]
    

from langchain_core.documents import Document

class ProductCombinedInformation:
    
    product: RAWProduct
    
    def __init__(self, product: RAWProduct) -> None:
        self.product = product
        
    def get_document(self):
        return Document(page_content=self.info(), id=str(self.product.id), metadata={
            "product_id": self.product.id,
            "product_asin": self.product.product_asin,
            "image_url": self.product.image_url,
            "name": self.product.name
        })
    
    
    def info(self):
        return (
            f"Product ID: {str(self.product.id).strip()} \n"
            f"Product Name: {str(self.product.name).strip()} \n"
            f"Product Description: {str(self.product.description).strip()} \n"
            f"Product Asin: {self.product.product_asin} \n"
            f"Overall Ratings {self.product.overall_ratings} \n"
            f"Total Customers that rated: {self.product.total_customers_that_rated} \n"
            f"Pric: {self.product.currency}{self.product.price} \n"   
        )
        
    def image_path(self):
        return f"{product_images_path}/{self.product.product_asin}.png"
    

In [4]:

product_summary_model_name = "llama3.1"

# product_summary_model_name = "gpt-4o-mini"
# image_description_model_name = "gpt-4o-mini"
# text_embedding_model_name = ""

### PID: Product Image Description

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_community.chat_models import ChatOllama
from PIL import Image
from IPython.display import HTML, display
import base64
from PIL import Image
import base64
from io import BytesIO

def encode_image(image_path: str):
    """Resizes the image to 150x150 pixels and returns the base64 string for the image"""
    
    # Open the image
    with Image.open(image_path) as img:
        # Resize the image to 150x150 pixels
        img = img.resize((150, 150))

        # Save the resized image to a BytesIO object (in-memory file)
        buffered = BytesIO()
        img.save(buffered, format="PNG")

        # Get the base64 encoded string
        return base64.b64encode(buffered.getvalue()).decode("utf-8")


def describe_image(img_base64, prompt, model_name: str, image_format: str = "png"):
        
    
    """
    Describes the image provided. The prompt is used to guide the model on what
    to do with the description and what it should do.

    Raises:
        ValueError: If the wrong model name is provided.
        Exception: If any error occur

    Returns:
        str: The description generated for the image
    """
    
    if model_name not in [ "llava", "gpt-4o", "gpt-4o-mini" ]:
        raise ValueError("Please provide either llava, gpt-4o or gpt-4o-mini")
    
    image_url = f"data:image/{image_format};base64,{img_base64}"
    
    if model_name.startswith("gpt-4o"):
        chat = ChatOpenAI(model=model_name, max_tokens=1024, temperature=0)
        msg = chat.invoke([
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url
                        }
                    }
                ]
            )
        ])
    elif model_name == "llava":
        chat = ChatOllama(model=model_name, num_ctx=1024)
        msg = chat.invoke([
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url
                        }
                    }
                ]
            )
        ])
    else:
        raise Exception("Unknown model name.")
        
    return msg.content


def plt_img_base64(img_base64):
  """Display base64 encoded string as image"""

  # Create an HTML img tag with the base64 string as the source
  image_html = f'<img width="200px" height="200px" src="data:image/jpeg;base64,{img_base64}" />'
  display(HTML(image_html))

### PCI: Product Combined Info

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnablePassthrough

def summarise_product_info(product_info: str, model_name: str):
    
    if model_name not in [ "llava", "gpt-4o", "gpt-4o-mini" , "llama3", "llama3.1", "phi3", "mistral" ]:
        raise ValueError("Please provide either llava, gpt-4o or gpt-4o-mini")
    
    prompt_template = ("You are an assistant tasked with summarizing tables and "
                    "text for retrieval. These summaries will be embedded and used " 
                    "to retrieve the raw text. Give a concise summary of the product. "
                    "Ensure you include important details "
                    "that is well optimized for retrieval.\n"
                    f"Product: {product_info}\n"
                    "Your response should start with the product name."
                    )
    
    prompt = ChatPromptTemplate.from_template(prompt_template)
    
    if model_name.startswith("gpt-4o"):
        model = ChatOpenAI(model=model_name, max_tokens=1024, temperature=0)
    else:
        model = ChatOllama(model=model_name, temperature=0)
    
        
    # { "product_info": RunnablePassthrough() } is same as lambda x: x
    # basically do not modify what the user entered, pass into the key product_info
    summarise_chain = { "product_info": RunnablePassthrough() } | prompt | model | StrOutputParser()
    summary = summarise_chain.invoke(product_info)
    return summary
    

In [7]:
max_products_per_category = 20

product_data.keys()

dict_keys(['MEN_FASHION_WITH_REVIEWS', 'SKIN_CARE_WITH_REVIEWS', 'WOMEN_FASHION_NO_REVIEWS'])

### Men Fashion

In [8]:
products = product_data['MEN_FASHION_WITH_REVIEWS']['products'][0:max_products_per_category]


# Remove Duplicates
filtered_products = []
checked_products = []

print("Before Removing Duplicates ", len(products))
for product in products:
    if product['product_asin'] not in checked_products:
        print(product['product_asin'])
        filtered_products.append(product)
        checked_products.append(product['product_asin'])

products = filtered_products

print("After Removing Duplicates ", len(products))

combined_product_infos = [ ProductCombinedInformation(product=RAWProduct.model_validate(product)) for product in products ]

# Summarise Product Info = PCIS
product_info_summaries = [summarise_product_info(product_info.info(), model_name=product_summary_model_name) for product_info in combined_product_infos]


# for pis in product_info_summaries:
#     print(pis, "\n\n")

Before Removing Duplicates  20
B0C812K6RR
B07BN6KCZT
B001BEAWXY
B07BSZXWWT
B01DBU6MQ6
B0BHMB8F2Q
B0BRNFVSCF
B098PCGH4F
B004LXWRH6
B09QRPLWJ3
B01B1D12B0
B09C5FHR99
B091SJ27ND
B09L7KYXL5
B07WZDCNSZ
B09HXLDG5B
After Removing Duplicates  16


In [9]:

# Describe the Product Image = PID
image_prompt = f"""
Describe and summarise the characteristics of the product you are looking at. Start your response with: `The image is a product ... `. Return back plain text no markdown.
"""

# image_description_model_name = "gpt-4o"
image_description_model_name = "llava"
image_prompt = f"""
Describe and summarise the characteristics of the product you are looking at. Start your response with: `The image is a product ... `. 
Give a short summary of what the product can be used for.
Return back plain text no markdown.
"""
product_image_descriptions = [describe_image(encode_image(image_path=product_info.image_path()), prompt=image_prompt, model_name=image_description_model_name) for product_info in combined_product_infos ]

for index, pid in enumerate(product_image_descriptions):
    print(index, pid, "\n\n")

0  The image is a product showcase of three pairs of shoes. The shoes are black and appear to be athletic or running shoes, with different color accents on the laces and other design details. They have a modern and sporty look, suitable for casual wear or exercise purposes.  


1  The image is a product advertisement featuring a person wearing a gray shirt and a brown jacket. The background is white, which puts the focus on the clothing. The person is standing upright with their arms at their sides, smiling towards the camera. They appear to be posing for the photo. The jacket has a collar, two chest pockets, and buttoned up at the front. It's designed to be worn over a shirt or sweater, offering a layering option for warmth during colder weather.  


2  The image is a product display of several pairs of men's underwear. These are brief-style boxer shorts, presented in a single package, with the brand name "Calvin Klein" visible on each set. The packaging suggests that these boxers mig

In [10]:
product_info_summaries[2], product_image_descriptions[2]

('Here is a concise summary of the product, optimized for retrieval:\n\n**Men Boxer Short Trunks Stretch Cotton Pack of 3**\n\n* **Product Details**: Calvin Klein boxer shorts with CK branding, designed for everyday wear and sports.\n* **Material Composition**: 95% Cotton, 5% Elastane (Spandex).\n* **Care Instructions**: Machine Washable.\n* **Key Features**:\n\t+ Comfortable fit due to stretchy fabric.\n\t+ Classic elastic waistband with logo.\n\t+ Soft and breathable cotton stretch material.\n* **Brand Information**: Calvin Klein lifestyle brand, known for minimalist aesthetics and timeless designs.\n* **Product Specifications**:\n\t+ Package Dimensions: 19.56 x 13.21 x 3.3 cm; 100 Grams.\n\t+ Date First Available: June 18, 2023.\n\t+ Manufacturer: Calvin Klein.\n\t+ Item Model Number: 0000U2662G.\n* **Customer Reviews**: 4.6 out of 5 stars (36,088 ratings).\n* **Price**: £28.0.\n\nThis summary includes the most important details about the product, making it easy to retrieve and comp

### Create the Multi Vector Retriever

In [11]:
!pip install --quiet langchain_chroma

DEPRECATION: Loading egg at /home/solomon/.local/lib/python3.11/site-packages/openapi_client-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /home/solomon/anaconda3/envs/ciam2rag/lib/python3.11/site-packages/imagebind-0.1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /home/solomon/anaconda3/envs/ciam2rag/lib/python3.11/site-packages/pytorchvideo-0.1.5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [12]:

# Vector Store for Document Embeddings
from langchain_chroma import Chroma
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from langchain_openai import OpenAIEmbeddings

from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

from chromadb.errors import InvalidDimensionException

def get_base_retriever(reset_collection: bool = False):
        
    # OpenCLIP: https://github.com/mlfoundations/open_clip
    # Larger and more performant
    # model_name = "ViT-g-14"
    # checkpoint = "laion2b_s34b_b88k"

    # # Smaller less performant
    # model_name = "ViT-B-32"
    # checkpoint = "laion2b_s34b_b79k"

    # # Default
    model_name = "ViT-H-14"
    checkpoint = "laion2b_s32b_b79k"

    text_embedding_model_name = "nomic-embed-text"

    # for nomic-embed, embed_instruction is `search_document` to embed documents for RAG and `search_query` to embed the question
    underlying_embedding = OllamaEmbeddings(model=text_embedding_model_name, embed_instruction="search_document", query_instruction="search_query")
    # underlying_embedding = OpenAIEmbeddings()
    reset_collection = True

    store = LocalFileStore("./embeddings-cache")

    cached_embedder = CacheBackedEmbeddings.from_bytes_store(
        underlying_embeddings=underlying_embedding, document_embedding_cache=store, namespace=underlying_embedding.model
    )


    collection_name = f"fashion_store_mrag_v_{underlying_embedding.model}"
    try:
        products_vectorstore = Chroma(
            collection_name=collection_name ,
            embedding_function=cached_embedder,
            # https://docs.trychroma.com/guides#changing-the-distance-function
            # Cosine, 1 means most similar, 0 means orthogonal, -1 means opposite
            collection_metadata={"hnsw:space": "cosine"}, # l2 is the default
            # embedding_function=OpenCLIPEmbeddings(model=None, preprocess=None, tokenizer=None, model_name=model_name, checkpoint=checkpoint)
        )
    except InvalidDimensionException:
        Chroma().delete_collection()
        products_vectorstore = Chroma(
            collection_name=collection_name ,
            embedding_function=cached_embedder,
            collection_metadata={"hnsw:space": "cosine"}, # l2 is the default
        )
        
    if reset_collection:
        products_vectorstore.reset_collection()
        
    return products_vectorstore


In [13]:
import json
import os
from pathlib import Path

images_path = "../initial-training-sets/datasets/images"

from langchain.retrievers.multi_vector import MultiVectorRetriever, SearchType
from langchain.storage import InMemoryStore
from langchain_core.documents import Document

from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=4000)

n_docs_to_retrieve = 15
relevant_threshold_score = 0.5

def create_multi_vector_retriever(
        embeddings_vectorsotre, 
        product_combined_info_summaries: List[str],  # PCIS -> Product Combined Info Summary
        product_image_descriptions: List[str],  # PID
        product_combined_infos: List[ProductCombinedInformation] # PCI objects
    ):
    
    # Initialise the document storage layer
    document_store = InMemoryStore()
    id_key = "product_id"
    
    retriever = MultiVectorRetriever(
        vectorstore=embeddings_vectorsotre,
        docstore=document_store,
        id_key=id_key,
        search_type=SearchType.similarity_score_threshold,
        search_kwargs={
            # below works only for similarity_score_threshold search_type
            "score_threshold": relevant_threshold_score, # close to 0 means most dissimilar, 1 means most similar
            "k": n_docs_to_retrieve
        }
    )
    
    # Helper function to add documents into the vector and the doument store
    def add_documents_with_splits(retriever: MultiVectorRetriever, summary_to_embeds: List[str], combined_product_infos: List[ProductCombinedInformation]):
        product_ids=[ str(product_info_obj.product.id) for product_info_obj in combined_product_infos ]
        data = list(zip(product_ids, summary_to_embeds, combined_product_infos))
               
        docs = []
        parent_docs_contents = []
        
        for single_item in data:
            product_id, content_to_embed, product_info = single_item
            for content_chunk in splitter.split_text(content_to_embed):
                docs.append(Document(
                    page_content=content_chunk,
                    metadata={
                        id_key: product_id,
                        "id": product_id,
                        "name": product_info.product.name,
                        "product_asin": product_info.product.product_asin
                    }
                ))
                parent_docs_contents.append(
                    Document(
                        page_content=product_info.info(),
                        metadata={
                            id_key: product_info.product.id,
                            "id": product_info.product.id,
                            "name": product_info.product.name,
                            "product_asin": product_info.product.product_asin
                        }
                    ) 
                )
        
        assert len(docs) == len(parent_docs_contents)
        
        retriever.vectorstore.add_documents(docs)
        retriever.docstore.mset(list(zip(product_ids, parent_docs_contents)))
        
        return retriever
    
    # Helper function to add documents into the vector and the doument store
    def add_documents_no_split(retriever: MultiVectorRetriever, summary_to_embeds: List[str], combined_product_infos: List[ProductCombinedInformation]):
        product_ids=[ str(product_info_obj.product.id) for product_info_obj in combined_product_infos ]
        data = list(zip(product_ids, summary_to_embeds, combined_product_infos))
               
        docs = []
        parent_docs_contents = [
                    Document(
                        page_content=product_info.info(),
                        metadata={
                            id_key: product_info.product.id,
                            "id": product_info.product.id,
                            "name": product_info.product.name,
                            "product_asin": product_info.product.product_asin
                        }
                    ) 
                    for product_info in combined_product_infos
        ]
        
        for single_item in data:
            product_id, content_to_embed, product_info = single_item
            docs.append(Document(
                page_content=content_to_embed,
                metadata={
                    id_key: product_id,
                    "id": product_id,
                    "name": product_info.product.name,
                    "product_asin": product_info.product.product_asin
                }
            ))
        
        assert len(docs) == len(parent_docs_contents)
        
        retriever.vectorstore.add_documents(docs, ids=product_ids )
        retriever.docstore.mset(list(zip(product_ids, parent_docs_contents)))
        
        return retriever
    
    if product_combined_info_summaries:
        add_documents_no_split(
            retriever=retriever,
            summary_to_embeds=product_combined_info_summaries,
            combined_product_infos=product_combined_infos
        )
        
    if product_image_descriptions:
        add_documents_no_split(
            retriever=retriever,
            summary_to_embeds=product_image_descriptions,
            combined_product_infos=product_combined_infos
        )
        
    return retriever
        
        

In [14]:
products_vectorstore = get_base_retriever(reset_collection=True)
retriever = create_multi_vector_retriever(
    embeddings_vectorsotre=products_vectorstore, 
    product_combined_info_summaries=product_info_summaries, 
    product_image_descriptions=product_image_descriptions,
    product_combined_infos=combined_product_infos, 
)

# retriever_query = "I dark coloured casual pants" # description only in the image
# retriever_query = "high tech glasses" # description only in the image
# retriever_query = "queen" # description only in the image
# retriever_query = "pants with zipper fly" # description only in the image

# docs = retriever.invoke(retriever_query)

# for doc in docs:
#     print(doc.page_content)
    

# print("----")
# # retriever.vectorstore.similarity_search_with_relevance_scores(retriever_query, k=3)
# docs = retriever.vectorstore.similarity_search_with_relevance_scores(retriever_query, k=n_docs_to_retrieve)
# for doc, score in docs:
#     print(score, score >= relevant_threshold_score , " - " , doc.metadata.get('product_asin') )


### Vanilla RAG: Synthesize Answer / Pass docs to LLM

In [30]:
from langchain_community.chat_models.ollama import ChatOllama

from langchain.output_parsers.pydantic import  PydanticOutputParser
from pydantic import BaseModel, Field

class ProductDetail(BaseModel):
  product_id: UUID = Field(description="ID of the product recommended to the user")
  detail: str = Field(description="Assistant message to the user about the product")
  title: str = Field(description="Price of the product")

class AssistantResponse(BaseModel):
  response: str = Field(description="The start of the assistant's answer.")
  products: List[ProductDetail] = Field(description="A collection of products being recommended for the user")


parser = PydanticOutputParser(pydantic_object=AssistantResponse)

answer_generation_model = "gpt-4o-mini"
# answer_generation_model = "llama3.1"
# answer_generation_model = "mistral"

if "gpt" not in answer_generation_model:
  model = ChatOllama(model=answer_generation_model, temperature=0, num_ctx=1024)
else:
  model = ChatOpenAI(model=answer_generation_model, temperature=0)


from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever

# Retrieval Chain: summarise history + current query/input to fetch relevant documents
history_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        SystemMessage(content="generate a search query to look up in order to get information relevant to the conversation."),
        ("user", "{input}"),
    ]
)
history_chain = create_history_aware_retriever(
  llm=model, retriever=retriever, prompt=history_prompt
)

# Answer Synthesising
system_message = """
                  \n{format_instructions}\n
                  
                  You work as a stylish and Gen-Z stylish who is working as a customer assistance in a fashion store.
                  You help customers to answer general questions about fashion and then tries to recommend a product 
                  if there is any that matches in the inventory below. If no inventory, do not recommend any product 
                  but you can provide general advice on fashion for them.
                  
                  <INVENTORY_START>
                  {context}
                  <INVENTORY_ENDS>
                  
                  Do not provide any link. Always reply as if you are talking to customer directly.
                  
                  In situation where there are multiple products in the inventory, explain how each of them can be helpful.
                  
                  There are times the inventory will be empty, assure the user we will get the product that would match their query if it is something a 
                  fashion store can provide as a service.
                  
                  All your response will be parsed as a JSON not markdown, which will fail if it is not in JSON format.
                """
                
                #   You should format your response in this format. 
                #   {{
                #     'response_summary': "",
                #     "products": [
                #       {{ "product_asin": "", "detail": "" }}
                #     ]
                #   }}
                  
                # """
format_instructions = parser.get_format_instructions()
prompt = ChatPromptTemplate.from_messages([
    ("system", system_message ),
    MessagesPlaceholder(variable_name="chat_history"), # empty for now
    ("human", "{input}" ),
])

# a chain that allows us pass a list of docs to a model
stuff_documents_and_prompts_chain = create_stuff_documents_chain(llm=model, prompt=prompt)



# a chain to use the retriever we have to retireve the relevant docs and then pass them to the chain that has the llm+prompt
vanilla_chain = create_retrieval_chain(
  retriever=history_chain, # We can either pass a retriever or a chain
  combine_docs_chain=stuff_documents_and_prompts_chain
)

# user_question = "The summer is here, what item can I buy to add to my accessory? Glasses, hats etc"
user_question = "The summer is here, what item can I buy to add to my accessory?"

# The documents fetched does not meet our threshold for relevance scores, so it wil be empty. Inventory is empty!
# docs = retriever.vectorstore.similarity_search_with_relevance_scores(user_question, k=n_docs_to_retrieve)
# for doc, score in docs:
#     print(score, score >= relevant_threshold_score , " - " , doc.metadata.get('product_asin'))


answer = vanilla_chain.invoke({"input": user_question, "chat_history": [], 
                       "format_instructions": format_instructions}
                      )
print(answer['answer'])
# import json
# json.loads(answer['answer'])

{
  "response": "Summer is the perfect time to elevate your accessory game! One great option is a stylish pair of sunglasses. They not only protect your eyes from harmful UV rays but also add a chic touch to your summer outfits.",
  "products": [
    {
      "product_id": "96f87d3b-5202-4ada-8b60-a1326719302f",
      "detail": "Men's Retro Driving Polarized Sunglasses with Al-Mg Metal Frame. These sunglasses are ultra-light and provide 100% UV400 protection, making them perfect for summer outings. They come with a vintage leather case and are designed for comfort and style.",
      "title": "Men's Retro Driving Polarized Sunglasses"
    }
  ]
}


#### What if we have multiple products?

In [34]:
user_question = "I need to buy trousers! I need like three."
# user_question = "I want to buy some new trousers, I need three"

# The documents fetched does not meet our threshold for relevance scores, so it wil be empty. Inventory is empty!
# docs = retriever.vectorstore.similarity_search_with_relevance_scores(user_question, k=n_docs_to_retrieve)
# for doc, score in docs:
#     print(score, score >= relevant_threshold_score , " - " , doc.metadata.get('product_asin'))


answer = vanilla_chain.invoke({"input": user_question, "chat_history": [], "format_instructions": format_instructions})
answer ['answer']

'{\n  "response": "I can help you with that! Here are some great trouser options from our inventory:",\n  "products": [\n    {\n      "product_id": "696b7628-954d-4303-b773-c81cd0ad687e",\n      "detail": "These slim fit trousers are stylish and comfortable, perfect for various occasions like business, casual outings, or even golf. They feature a stretch material for optimal comfort and mobility.",\n      "title": "Comfortable and Modern Mens Trousers with Stretch"\n    }\n  ]\n}'

#### Vanilla RAG: Image + Text Customer Prompt

Implement how we can query the RAG system with a text and a picture

In [35]:
product_asin = "B07BN6KCZT" # grey shirt
product_asin = "B01B1D12B0" # sun glasses
query_image_path = f"../initial-training-sets/MEN_FASHION_WITH_REVIEWS/images/{product_asin}.png"

# Encode the image
query_image_encoded = encode_image(query_image_path)
# Summarise the image
describe_query_image_prompt = """Describe what you are seeing to enable us to retrieve an items that resembles this product"""
# Compose User Prompt and Pass the image+prompt to the conversational chain
query_image_description = describe_image(query_image_encoded, prompt=describe_query_image_prompt, model_name=image_description_model_name)

print("Customer Image Described", query_image_description)

# Test this with out vector store
# image_related_docs = retriever.vectorstore.similarity_search_with_relevance_scores(query_image_description, k=n_docs_to_retrieve)
# for doc, score in image_related_docs:
#     print(score, doc.page_content)
    

customer_prompt = "I am looking to buy a set of glasses that looks like this and a trouser that goes well with Suits."
customer_prompt = "I am looking to buy a set of glasses that looks like this and a trouser that goes well with it"

customer_prompt_formatted = f"""
Customer Message: {customer_prompt}.
<DESCRIPTION_START>{query_image_description}<DESCRIPTION_ENDS>
"""

vanilla_chain.invoke({"input": customer_prompt_formatted, "chat_history": [], "format_instructions": format_instructions})

Customer Image Described  This is an image of a pair of sunglasses. The glasses have a reflective surface on the lenses, suggesting that they might be photochromic or polarized, which can change tint based on light conditions. They feature a design with a two-tone frame, possibly a gradient of colors, and what appears to be a double bridge structure, with one arm being noticeably longer than the other. The temples of the glasses have a patterned design, and the brand logo is visible on the right lens. The glasses also have a slight curve along their top edge, which could contribute to improved ventilation for comfort during wear. 
0.8136507868766785  The image is a product photograph of a pair of sunglasses. These sunglasses feature a folding design, which allows them to be easily transported or stored. The frame appears to have a tortoiseshell pattern and is likely made of plastic or acetate. The arms of the sunglasses are long and thin, suggesting they are designed for comfort behind

{'input': '\nCustomer Message: I am looking to buy a set of glasses that looks like this and a trouser that goes well with it.\n<DESCRIPTION_START> This is an image of a pair of sunglasses. The glasses have a reflective surface on the lenses, suggesting that they might be photochromic or polarized, which can change tint based on light conditions. They feature a design with a two-tone frame, possibly a gradient of colors, and what appears to be a double bridge structure, with one arm being noticeably longer than the other. The temples of the glasses have a patterned design, and the brand logo is visible on the right lens. The glasses also have a slight curve along their top edge, which could contribute to improved ventilation for comfort during wear. <DESCRIPTION_ENDS>\n',
 'chat_history': [],
 'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "descrip

### Decomposition

### Available Products

In [18]:
product_info_summaries

['**Safety Trainers Men Women Steel Toe Cap Trainers Safety Shoes Lightweight Work Shoes Safety Boots Industrial Protective Shoes**\n\n**Summary:**\n\n* **Product Type:** Safety Trainers, Steel Toe Cap, Lightweight Work Shoes\n* **Key Features:**\n\t+ Breathable upper with mesh hole design for heat dispersion and odor control\n\t+ Anti-smashing steel toe cap with 200J force absorption and 15000N compression resistance\n\t+ Puncture-proof Kevlar midsole with 1200N force resistance\n\t+ Ultra-lightweight, wear-resistant non-slip TPR soles\n* **Industries:** Construction site, exploitation site, forging workshop, manufacturing, logistics, warehouse, factory, gardening\n* **Product Details:**\n\t+ Package Dimensions: 28.8 x 17.2 x 10.4 cm; 740 Grams\n\t+ Date First Available: June 13, 2023\n\t+ ASIN: B0C812JJB8\n\t+ Department: Unisex\n* **Customer Reviews:** 4.3 out of 5 stars (157 ratings)\n* **Price:** £16.99',
 "Here is a concise summary of the product:\n\n**Men's Regular-Fit Long-Slee

#### Without Image

In [106]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """
Imaging you are a customer that walks into a Fashion Store. You will like to purchase or make inquiry. Your input question is {question}.\n
Identify the products being asked for and break them down into customer sub-statements standing in isolation\n

Unless there is a specific attribute of characteristics of the user, do not generate personalise sub-statements\n

Examples:
Main Question: I need a nice dress I can wear to a suit.

System
The customer is looking for a suit.
What goes well with a suit for man? Belts, blazers, wrist watches, ties, corporate shoes,
I will generate question for each of this product

Output 
Customer: I am looking for a belt to wear with my suit
Customer: I am looking for wrist watches to wear with my suit
Customer: I am looking for ties to wear with my suit
Customer: I am looking for shoes that will go well with my suit

No markdown, just plain text


Think carefully, and identify at least 10 products that could match this input question.
"""

# template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
# The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
# Do not generate questions are that targetted to any brand. \n
# Generate multiple search queries related to: {question} \n
# Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)



from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# llm = ChatOllama(model="llama3.1")

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("Customer:")))

# Run
question = "I want to buy long sleeve shirt and jeans, with a tie for my new suit I have at home"
question = "As a man, I am currently completing my masters program in the UK. I have my graduation by next year January. I need a recommendation of dress to wear."
question = "I'm planning a day out in the city where I'll be walking a lot. I need a combination that is stylish and comfortable?"

questions = generate_queries_decomposition.invoke({"question":question})
if len(questions) > 3:
    questions = questions[1:len(questions)]
    
questions

[' I am looking for comfortable shoes suitable for walking around the city.  \n',
 ' I am looking for stylish sneakers that I can wear for a day out.  \n',
 ' I am looking for breathable tops that are both stylish and comfortable.  \n',
 ' I am looking for lightweight pants or shorts that are easy to move in.  \n',
 ' I am looking for a fashionable yet functional backpack for my day out.  \n',
 ' I am looking for a stylish hat to protect me from the sun.  \n',
 ' I am looking for sunglasses that are both trendy and provide UV protection.  \n',
 ' I am looking for a light jacket or cardigan for layering.  \n',
 ' I am looking for comfortable leggings that I can wear while walking.  \n',
 ' I am looking for accessories like a crossbody bag that is easy to carry.  ']

In [107]:
# for q in questions:
#     # The documents fetched does not meet our threshold for relevance scores, so it wil be empty. Inventory is empty!
#     docs = retriever.vectorstore.similarity_search_with_relevance_scores(q, k=n_docs_to_retrieve)
#     for doc, score in docs:
#         print(q, score, score >= relevant_threshold_score , " - " , doc.metadata.get('product_asin'), doc.metadata.get("name"))

### Answer recursively (Does not work for our use case)
- Chain-of-Thought Reasoning
 https://arxiv.org/abs/2212.10509

- LEAST-TO-MOST PROMPTING ENABLES COMPLEX REASONING IN LARGE LANGUAGE MODELS
 https://arxiv.org/pdf/2205.10625

In [108]:
# questions

In [109]:
# from operator import itemgetter
# from langchain_core.output_parsers import StrOutputParser

# # Prompt
# template = """Here is the customer question you need to answer:

# \n --- \n {question} \n --- \n

# Here is any available background question + answer pairs:

# \n --- \n {q_a_pairs} \n --- \n

# Here is additional context relevant to the question: 

# \n --- \n {context} \n --- \n

# Use the above context and any background question + answer pairs to answer the customer question: \n 
# Include the product id in the answer
# \n
# {question}
# """

# decomposition_prompt = ChatPromptTemplate.from_template(template)

# def format_qa_pair(question, answer):
#     """Format Q and A pair"""
    
#     formatted_string = ""
#     formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
#     return formatted_string.strip()

# # llm
# # model_name = "gpt-3.5-turbo"
# # model_name = "llama3.1"
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# # llm = ChatOpenAI(model_name=model_name, temperature=0)
# # llm = ChatOllama(model=model_name, temperature=0)

# q_a_pairs = ""
# for q in questions:
    
#     rag_chain = (
#     {"context": itemgetter("question") | retriever, 
#      "question": itemgetter("question"),
#      "q_a_pairs": itemgetter("q_a_pairs")} 
#     | decomposition_prompt
#     | llm
#     | StrOutputParser())

#     answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
#     q_a_pair = format_qa_pair(q,answer)
#     q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

### Answer individually

In [110]:
# Answer each sub-question individually 

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
# llm = ChatOllama(model="llama3.1", temperature=0)

template_str = """
You work as a fashion store customer support, use the inventory below to answer this question {question}\nContext: {context}.

Output:

```json
[
    {{
        "name": "",
        "product_asin": "",
        "product_id": "",
    }}
]

```

If no item
{{
    "name": 'null'
    "product_asin": 'null'
    "product_id": null,
}}

All your answers must be from the inventory.
"""
prompt_template = ChatPromptTemplate.from_template(template_str)
def retrieve_and_rag(question,prompt,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        
        
        # # Use retrieved documents and sub-question in RAG chain
        answer = (prompt| llm | JsonOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        
        rag_results.append(answer)
    
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_template, generate_queries_decomposition)

In [111]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# # Prompt
# template = """
# As an empathy customer support assistant for a fashion store, you are being presented with a list customers questions and the products we have in the inventory.\n

# INVENTORY_START\n
# {context}\n
# INVENTORY_ENDS \n

# Use these to synthesize an answer to the question: {question}\n

# Do not mention the name of the product but instead the `product_asin`, prefixed and suffixed with #\n
# Your message should contain more informatino that will help the customer to make a decision quickly.\n

# Output
# ```json
# {{   
#     "assistant_message": "We have the #PRODUCT_ASIN# which goes perfectly with your blouse.",
#     "products": [    
#         {{
#             "name": "The <PRODUCT_NAME> is the best",
#             "product_asin": "",
#             "detail": "This <product_name> goes perfectly with the other product because it has these attributes"
#         }}
#     ]
# }}
# ```

# All products and product asin must come from the inventory above.

# """

# prompt = ChatPromptTemplate.from_template(template)


# Answer Synthesising
system_message = """
    \n{format_instructions}\n
    
    You work as a stylish and Gen-Z stylish who is working as a customer assistance in a fashion store.
    You help customers to answer general questions about fashion and then tries to recommend a product 
    if there is any that matches in the inventory below. If no inventory, do not recommend any product 
    but you can provide general advice on fashion for them.
    
    <INVENTORY_START>
    {context}
    <INVENTORY_ENDS>
    
    Do not provide any link. Always reply as if you are talking to customer directly.
    
    In situation where there are multiple products in the inventory, explain how each of them can be helpful and can be combined.
    
    There are times the inventory will be empty, assure the user we will get the product that would match their query if it is something a 
    fashion store can provide as a service.
    
    All your response will be parsed as a JSON not markdown, which will fail if it is not in JSON format.
"""
format_instructions = parser.get_format_instructions()
decomposition_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message ),
    MessagesPlaceholder(variable_name="chat_history"), # empty for now
    ("human", "{input}" ),
])


final_rag_chain = (
    decomposition_prompt
    | llm
    | JsonOutputParser()
)

answer = final_rag_chain.invoke({"context":context,"input":question, "chat_history": [], "format_instructions": format_instructions})
print(answer)

{'response': "For a stylish and comfortable day out in the city, I recommend a combination of a hoodie and lightweight shoes. The Essentials Hoodie is perfect for keeping you warm while being stylish, and you can pair it with the Lightweight Safety Shoes which are breathable and designed for comfort during long walks. Additionally, you could consider wearing the Men's Regular-Fit Long-Sleeve Two-Pocket Flannel Shirt for a more layered look, should the weather require it. This way, you'll be both fashionable and comfortable throughout your day.", 'products': [{'product_id': '200a76d1-3a22-4d92-ae0a-15ec52eca9ca', 'detail': 'Lightweight Safety Shoes Men Women Steel Toe Cap Trainers Work Shoes Breathable Non Slip Industrial Work Boots are designed for comfort and support during long walks.', 'title': 'Lightweight Safety Shoes'}, {'product_id': '5291d426-5e8f-4b59-a770-5ce9336b7710', 'detail': 'The Essentials Hoodie is a stylish option that provides warmth and comfort, making it a great ch

## Test Vanilla RAG on our question

In [112]:
# a chain to use the retriever we have to retireve the relevant docs and then pass them to the chain that has the llm+prompt
vanilla_chain = create_retrieval_chain(
  retriever=history_chain, # We can either pass a retriever or a chain
  combine_docs_chain=stuff_documents_and_prompts_chain
)


answer = vanilla_chain.invoke({"input": question, "chat_history": [], 
                       "format_instructions": format_instructions}
                      )
print(answer['answer'])


{
  "response": "For a day out in the city where you'll be walking a lot, it's important to choose footwear that is both stylish and comfortable. Here are a couple of recommendations from our inventory that would be perfect for your needs:",
  "products": [
    {
      "product_id": "200a76d1-3a22-4d92-ae0a-15ec52eca9ca",
      "title": "Lightweight Safety Shoes Men Women Steel Toe Cap Trainers Work Shoes",
      "detail": "These safety shoes are not only stylish but also incredibly comfortable. They feature a breathable design to keep your feet cool, a lightweight construction for ease of movement, and a non-slip sole for safety. Perfect for long walks in the city!"
    },
    {
      "product_id": "b9b7b87e-0786-4842-9d32-252aec8eb314",
      "title": "Safety Trainers Men Women Steel Toe Cap Trainers Safety Shoes",
      "detail": "These trainers are designed with a breathable upper and a lightweight feel, making them ideal for walking. They also have a stylish appearance and provide